<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/ALBERT_for_SNLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%load_ext google.colab.data_table
%tensorflow_version 2.x
from google.colab import drive #drive.flush_and_unmount()
from os import chdir
import numpy as np
import pandas as pd
drive.mount('/content/drive')
chdir('/content/drive/My Drive/CSIT Internship/code')

import torch
from torch import nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#!pip install transformers

TensorFlow 2.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import json
data = []
with open('./snli_1.0/snli_1.0_train.jsonl','r') as f:
    for line in f.readlines():
        data.append(json.loads(line))
df = pd.DataFrame(data)

In [0]:
import itertools
import operator
def most_common(L):
  # get an iterable of (item, iterable) pairs
  SL = sorted((x, i) for i, x in enumerate(L))
  # print 'SL:', SL
  groups = itertools.groupby(SL, key=operator.itemgetter(0))
  # auxiliary function to get "quality" for an item
  def _auxfun(g):
    item, iterable = g
    count = 0
    min_index = len(L)
    for _, where in iterable:
      count += 1
      min_index = min(min_index, where)
    # print 'item %r, count %r, minind %r' % (item, count, min_index)
    return count, -min_index
  # pick the highest-count/earliest item
  return max(groups, key=_auxfun)[0]

def fixEmpty(r):
    r.gold_label = most_common(r.annotator_labels)
    return r
df[df.gold_label == '-'] = df[df.gold_label == '-'].apply(fixEmpty,axis=1)

In [0]:
from transformers import AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer,AdamW
pretrained = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(pretrained)
#config = AlbertConfig.from_pretrained(pretrained)
model = AlbertForSequenceClassification.from_pretrained(pretrained)
model._modules['classifier'] = nn.Linear(in_features=768,out_features=3,bias=True)
model.config.num_labels = 3
model.num_labels = 3

classlist = {'contradiction':0,
             'entailment':1,
             'neutral':2}

In [0]:
options = {
    'max_length':                   512,            #causes overflow
    'stride':                       0,              #how many tokens before to start from in a overflow
    'add_special_tokens':           True,           #Adds [CLS] & [SEP]
    'truncation_strategy':          'only_second',  #Only body of article is cut
    'return_token_type_ids':        True,           #when using pairs, adds the type indicator
    'return_overflowing_tokens':    False,          #allows for overflowing tokens
    'return_special_tokens_mask':   True,           #masking tokens
    'return_tensors':               'pt'            #return as torch tensor
}
pad_c = tokenizer.special_tokens_map['pad_token']
sep_c = tokenizer.special_tokens_map['sep_token']
options['max_length'] += 1

def processInput(raw_batch):
    id_batch = []
    type_batch = []
    mask_batch = []
    label_batch = []
    for sent1,sent2,label in raw_batch:
        enc = tokenizer.encode_plus(sent1,sent2+sep_c+pad_c*options['max_length'],**options)

        token_id = enc['input_ids'][0][:options['max_length']-1]
        token_type = enc['token_type_ids'][0][:options['max_length']-1]
        mask = token_id.clone()
        mask[mask != 0] = 1

        id_batch.append(token_id.to(device))
        type_batch.append(token_type.to(device))
        mask_batch.append(mask.to(device))
        label_batch.append(classlist[label])
    return (torch.stack(id_batch),torch.stack(type_batch),torch.stack(mask_batch),torch.tensor(label_batch,device=device))
#processInput(df.head(5).apply(lambda row: (row.sentence1,row.sentence2,row.gold_label),axis=1))

In [0]:
import random
seed = 666
random.seed(seed)
batch_size = 16

def train(model,data,epochs=1):
    optimizer = AdamW(model.parameters())
    head_mask = [0]+ [1] * (model.config.num_attention_heads-1)
    model.train()
    for i in range(epochs):
        print('EPOCH %d'%i)
        loader = data.sample(frac=1,random_state=seed).reset_index(drop=True).groupby(np.arange(len(data))//batch_size)
        tr_loss = 0
        print('Total batches: %d'%len(loader))
        for g, df in loader:
            print('Batch %d'%g)
            id_batch,type_batch,mask_batch,label_batch = processInput(df.apply(lambda row: (row.sentence1,row.sentence2,row.gold_label),axis=1))
            random.shuffle(head_mask)
            headm = torch.tensor(head_mask,device=device)

            model.zero_grad()
            optimizer.zero_grad()

            loss, logits = model(input_ids=id_batch,
                                attention_mask=mask_batch,
                                token_type_ids=type_batch,
                                labels=label_batch,
                                head_mask=headm)[:2]
            
            loss.backward()
            tr_loss += loss.item()
            optimizer.step()
            print('Loss %f'%(tr_loss/(g+1)))

In [0]:
train(model,df)

EPOCH 0
Total batches: 34385
Batch 0
Loss 1.043603
Batch 1
Loss 1.127185
Batch 2
Loss 1.156964
Batch 3
Loss 1.145500
Batch 4
Loss 1.225963
Batch 5
Loss 1.219158
Batch 6
Loss 1.231108
Batch 7
Loss 1.241617
Batch 8
Loss 1.243058
Batch 9
Loss 1.298170
Batch 10
Loss 1.296156
Batch 11
Loss 1.278732
Batch 12
Loss 1.268191
Batch 13
Loss 1.255250
Batch 14
Loss 1.238127
Batch 15
Loss 1.248640
Batch 16
Loss 1.258103
Batch 17
Loss 1.260792
Batch 18
Loss 1.249558
Batch 19
Loss 1.244225
Batch 20
Loss 1.233132
Batch 21
Loss 1.224359
Batch 22
Loss 1.230350
Batch 23
Loss 1.230384
Batch 24
Loss 1.227822
Batch 25
Loss 1.223148
Batch 26
Loss 1.219139
Batch 27
Loss 1.216233
Batch 28
Loss 1.213799
Batch 29
Loss 1.211235
Batch 30
Loss 1.213236
Batch 31
Loss 1.217097
Batch 32
Loss 1.219258
Batch 33
Loss 1.214717
Batch 34
Loss 1.213100
Batch 35
Loss 1.213830
Batch 36
Loss 1.212602
Batch 37
Loss 1.209842
Batch 38
Loss 1.207621
Batch 39
Loss 1.204586
Batch 40
Loss 1.200721
Batch 41
Loss 1.198711
Batch 42
Loss 1